# Standard packages

In [1]:
import os
import pandas as pd
import time

# Specific installs

Contextfold is available as a Java package.

In [2]:
method_name = "contextFold"

In [3]:
url   = 'https://www.cs.bgu.ac.il/~negevcb/contextfold/'
fname = 'ContextFold_1_00.zip'

!wget -q {url+fname}                 # download zip file
!unzip {fname} > /dev/null           # decompress downloaded file
!mv {os.path.splitext(fname)[0]}/* . # move decompressed files to current dir 

# S. cerevisiae (sce) 18 long non-coding RNA dataset
Data source: https://genie.weizmann.ac.il/pubs/PARS10/data/sce_genes_folded.tab.gz

In [4]:
gh_path = "https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/master/data/"
sce = pd.read_csv(gh_path + "sce_genes_folded.tab", delimiter='\t', 
                  header=None, index_col=0, 
                  names=("Gene ID", "sequence", "PARS-assisted folding"))

In [5]:
# Sequences to process
yeast18lnc = ["snR81", "snR34", "snR43", "snR44",  "snR31",  "snR10",
              "snR63", "snR11", "snR82", "snR17b", "snR17a", "snR37",
              "SCR1",  "SRG1",  "snR19", "snR30",  "LSR1",   "TLC1"]

# Compute structures

In [6]:
def run_folding(fasta_name):
  out_file_name = fasta_name + '.pred'

  # make prediction
  os.system(f"java -cp bin contextFold.app.Predict in:{fasta_name} >/dev/null 2>&1")
  
  # remove last emtpy line
  os.system(f"head -n -1 {out_file_name} > {out_file_name + '.tmp'}")
  os.system(f"mv {out_file_name + '.tmp'} {out_file_name}")
  
  return out_file_name

In [7]:
out_fasta_name = method_name + "_yeast18"
if os.path.exists(out_fasta_name + ".fasta"): os.remove(out_fasta_name + ".fasta")

lnc_ids = yeast18lnc
print("   \t lnc \t len \t time")
for i, lnc in enumerate(lnc_ids): 

  start_time = time.time()
  seq = sce.loc[lnc]["sequence"]
  print(f"{i+1}/{len(lnc_ids)}\t{lnc} \t {len(seq)}", end='\t')

  # Write a one-sequence fasta
  with open("tmp.fasta", "w") as ofile: 
    ofile.write(f">{lnc}\n{seq}\n")
  
  dot_file_name = run_folding("tmp.fasta")

  # Concatenate outputs
  os.system("cat " + dot_file_name + " >> " + out_fasta_name + ".fasta") 

  print(f"{time.time() - start_time: .1f} s")

   	 lnc 	 len 	 time
1/18	snR81 	 201	 1.5 s
2/18	snR34 	 203	 0.5 s
3/18	snR43 	 209	 0.7 s
4/18	snR44 	 211	 1.1 s
5/18	snR31 	 225	 1.1 s
6/18	snR10 	 245	 1.1 s
7/18	snR63 	 255	 1.2 s
8/18	snR11 	 258	 1.1 s
9/18	snR82 	 268	 1.1 s
10/18	snR17b 	 332	 1.4 s
11/18	snR17a 	 333	 1.7 s
12/18	snR37 	 386	 2.1 s
13/18	SCR1 	 522	 2.0 s
14/18	SRG1 	 551	 1.4 s
15/18	snR19 	 568	 1.3 s
16/18	snR30 	 606	 1.5 s
17/18	LSR1 	 1175	 3.4 s
18/18	TLC1 	 1301	 4.0 s
